In [ ]:
import numpy as np
from isaacgym import gymapi, gymtorch
from isaacgym.torch_utils import *

In [ ]:
from franka_panda_env import FrankaPandaEnv
from motion_primitive import MotionPrimitive

In [ ]:
def wrap_angle(angle):
    """ Wrap an angle to the range [-pi, pi] """
    return angle - 2 * torch.pi * torch.floor((angle + torch.pi) / (2 * torch.pi))

In [ ]:
env = FrankaPandaEnv(use_gpu=True, use_gpu_pipeline=False)

In [ ]:
class EnvWrapper:
    def __init__(self)

In [ ]:
motion_primitive = MotionPrimitive(num_envs=env.num_envs)

In [ ]:
env.reset_pose("default")
env._refresh()
for i in range(10000):
    # if i % 100 == 0:
    #     env.reset_pose()
    #     env.reset_cube()
    env._refresh()
    dpose = torch.zeros((env.num_envs, 7))
    dpose[:, :3] = torch.Tensor([0.0, 0.2, 0.025]) - env.states["eef_pos"]

    target_quat = torch.Tensor([1, 0, 0, 0])
    current_quat = env.states["eef_quat"]
    diff_quat = quat_mul(
        target_quat.unsqueeze(0).expand_as(current_quat),
        quat_conjugate(current_quat),
    )
    euler_angles = get_euler_xyz(diff_quat)
    dpose[:, 3], dpose[:, 4], dpose[:, 5] = normalize_angle(euler_angles[0]), normalize_angle(euler_angles[1]), normalize_angle(euler_angles[2])
    dpose[:, -1] = -1
    
    # actions = motion_primitive.sample_actions()
    # dpose[:, :2] = actions * 0.3
    # dpose[:, 2] *= 3.0
    env.pre_physics_step(dpose)
    env.step_physics()
    env.gym.refresh_rigid_body_state_tensor(env.sim)
    env.step_rendering()
    # env.gym.write_viewer_image_to_file(env.viewer, f'./images/{i}.png')
    # break

In [ ]:
env.gym.get_sim_actor_count(env.sim) // env.num_envs

In [ ]:
for i in range(1000):
    env.step_physics()
    env.step_rendering()

In [ ]:
viewer.